In [20]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pickle
from tqdm.notebook import tqdm<<<<<
tqdm.pandas()

In [7]:
with open('data/clean/twitter/full_tweet_df_19-09-22.p', 'rb') as p:
    tweet_df = pickle.load(p)

In [13]:
ad_files = ['politician_ads_v1']#, 'party_ads_v1']
ad_dfs = []

for ad_file in ad_files:
    with open(f'data/clean/meta/{ad_file}.p', 'rb') as p:
        ad_dfs.append(pickle.load(p))

ad_df = pd.concat(ad_dfs).reset_index(drop = True)

In [15]:
print(ad_df['page_name'].head())
print(tweet_df['screen_name'].head())

0    Andreas Steenberg
1    Andreas Steenberg
2    Andreas Steenberg
3    Andreas Steenberg
4    Andreas Steenberg
Name: page_name, dtype: object
0    AndersKuhnau
1    AndersKuhnau
2    AndersKuhnau
3    AndersKuhnau
4    AndersKuhnau
Name: screen_name, dtype: object


In [57]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=75, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].progress_apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].progress_apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [58]:
twitter_names = tweet_df[['screen_name']].drop_duplicates()
meta_names = ad_df[['page_name']].drop_duplicates()

In [59]:
fuzzy_test = fuzzy_merge(twitter_names, meta_names, 'screen_name', 'page_name')

  0%|          | 0/638 [00:00<?, ?it/s]

  0%|          | 0/638 [00:00<?, ?it/s]

In [61]:
fuzzy_test.loc[fuzzy_test['matches'] != ''].tail(50)

,screen_name,matches
4032,simonkollerup,Simon Kollerup
7369,stinuslindgreen,Stinus Lindgreen - Radikale Venstre
10569,KirstenNormann,Kirsten Normann Andersen
27849,MadsAndersenC,Hans Andersen
28017,SorenPape,Søren Pape Poulsen
30838,SorenEggeRasmus,Søren Egge Rasmussen
33595,RasmusPrehn,Rasmus Prehn
36796,MarieKrarup,Marie Krarup
39173,AlexVanopslagh,Alex Vanopslagh
51719,Kristianthdahl,Kristian Thulesen Dahl


In [5]:
fuzz.ratio('Søren Pape Poulsen', 'Søren Pape')

71

In [ ]:
p